In [1]:
import os
import pandas as pd
import numpy as np
import geocoder
import feather
import re
from collections import Counter
import requests
from bs4 import BeautifulSoup
import time
from sklearn.mixture import GaussianMixture as GMM
from tqdm import tqdm_notebook as tqdm

In [2]:
train = pd.read_csv('../input/train.csv')
target = train["賃料"]
train = train.drop("賃料", axis=1)
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)

In [3]:
org_columns = all_df.columns
org_columns

Index(['id', '所在地', 'アクセス', '間取り', '築年数', '方角', '面積', '所在階', 'バス・トイレ', 'キッチン',
       '放送・通信', '室内設備', '駐車場', '周辺環境', '建物構造', '契約期間'],
      dtype='object')

In [4]:
all_df.isnull().sum()

id            0
所在地           0
アクセス          0
間取り           0
築年数           0
方角         5557
面積            0
所在階           1
バス・トイレ      649
キッチン       1256
放送・通信      6894
室内設備        155
駐車場       10643
周辺環境      18673
建物構造          0
契約期間      14101
dtype: int64

In [5]:
all_df.shape

(62732, 16)

In [6]:
"""記入ミスの修正"""
def modify_miss(df_input):
    df_input.loc[df_input["間取り"].str.contains("11R"), "間取り"] = "1R"
    
    df_input.loc[df_input["築年数"].str.contains("520年5ヶ月"), "築年数"] = "52年5ヶ月"
    
    df_input.loc[df_input["築年数"].str.contains("1019年7ヶ月"), "築年数"] = "19年7ヶ月"
    
    df_input.loc[df_input["面積"]=="430.1m2", "面積"] = "43.01m2"
    #idx = df_input["築年数"].str.contains("1019年7ヶ月")
    #df_input["築年数"][idx] = "19年7ヶ月"    
    df_input.loc[df_input["面積"]==("1m2"), "面積"] = "10m2"
    df_input.loc[df_input["面積"]==("5.83m2"), "面積"] = "58.3m2"
    
    "三田線 三田(兵庫県)駅 徒歩14分"
    
    return df_input

def modify_rent(target, df_input):
    """ id == 5776 で賃料の0が一つ多い"""
    target = np.where(df_input["id"]==5776, 120350, target)
    
    return target


#all_df = modify_miss(all_df)
target = modify_rent(target, train)

In [7]:
def count_encoding(df_input,column):
    df_out = pd.DataFrame()
    temp = df_input[column].value_counts().to_dict()
    df_out[f'count_{column}'] = df_input[column].map(temp)
    return df_out

In [8]:
def element_count_feature(df_input):
    """要素数数え"""
    df_out = pd.DataFrame()
    df_out["num_of_NULL"] = df_input.isnull().sum(axis=1)
    
    df_out["周辺環境_element_counts"] = df_input["周辺環境"].str.count("【").fillna(0)
    df_out["室内設備_element_counts"] = df_input["室内設備"].str.count("\t").fillna(-1)+1
    df_out["バス・トイレ_element_counts"] = df_input["バス・トイレ"].str.count("\t").fillna(-1)+1
    df_out["キッチン_element_counts"] = df_input["キッチン"].str.count("\t").fillna(-1)+1
    df_out["放送・通信_element_counts"] = df_input["放送・通信"].str.count("\t").fillna(-1)+1
    
    df_out["設備和"] = df_out[[col for col in df_out.columns if col != "num_of_NULL"]].sum(axis=1)
    
    df_count = pd.concat([count_encoding(df_out, column) for column in df_out.columns], axis=1)
    df_out = pd.concat([df_out,df_count], axis=1)
    return df_out

In [9]:
def clean_parking(text):
    obj = ["駐車場", "バイク置き場", "駐輪場"]
    if type(text)==float:
        return np.full(3, np.nan)
    else:
        text_list = text.split("\t")
        out_array = [np.nan, np.nan, np.nan]
        tmp_array = [(100, "駐車場"), (100, "バイク置き場"), (100, "駐輪場")]
        for i in range(len(obj)):
            if obj[i] in text_list:
                tmp_array[i] = (text_list.index(obj[i]), str(obj[i]))
                
        tmp_array.sort()
        if tmp_array[0][0] == 100:
            return out_array
        
        else:
            if tmp_array[1][0] != 100:
                out_array[obj.index(tmp_array[0][1])] = text_list[tmp_array[0][0]:tmp_array[1][0]]
            else:
                out_array[obj.index(tmp_array[0][1])] = text_list[tmp_array[0][0]:]
                return out_array
            
            if tmp_array[2][0] != 100:
                out_array[obj.index(tmp_array[1][1])] = text_list[tmp_array[1][0]:tmp_array[2][0]]
                out_array[obj.index(tmp_array[2][1])] = text_list[tmp_array[2][0]:]
            else:
                out_array[obj.index(tmp_array[1][1])] = text_list[tmp_array[1][0]:]
                return out_array


            return out_array
#b = all_df["駐車場"].apply(lambda x:clean_parking(x)).apply(pd.Series)

In [10]:
def list_unfold(text, col_name):
    if type(text)==float:
        return np.nan
    else:
        text = ",".join(text)
        text = re.sub(col_name+",", "", text)
        text = re.sub(",", "", text)
        return text

In [11]:
def feat(text):
    if type(text)==float:
        return np.full(3, np.nan)
    else:
        out_array = np.full(3, np.nan)
        text_yen = text.split("円")[0]
        try:
            out_array[1] = int(re.sub("[^0-9]", "", text_yen))
        except:
            pass
        
        if "無" in text_yen:
            out_array[0] = 0
        elif "近隣" in text_yen:
            out_array[0] = 1
        elif "有" in text_yen:
            out_array[0] = 2
            out_array[2] = 0
        try:
            text_meter = text.split("距離")[1].split("m")[0]
            out_array[2] = int(re.sub("[^0-9]", "", text_meter))
        except:
            pass
        return out_array

In [12]:
def parking_feature(df_input):
    tmp_df = df_input["駐車場"].apply(lambda x:clean_parking(x)).apply(pd.Series)
    tmp_df.columns = ["駐車場", "バイク置き場", "駐輪場"]
    for col in tmp_df.columns:
        tmp_df[col] = tmp_df[col].apply(lambda x:list_unfold(x, col))
    
    
    columns = ["駐車場", "バイク置き場", "駐輪場"]
    
    col_name = ["駐車場_有無", "駐車場_円", "駐車場_距離", "バイク置き場_有無", 
                "バイク置き場_円", "バイク置き場_距離", "駐輪場_有無", "駐輪場_円", "駐輪場_距離"]

    df_car = pd.DataFrame(tmp_df["駐車場"].apply(lambda x:feat(x)).apply(pd.Series))
    df_car.columns = ["駐車場_有無", "駐車場_円", "駐車場_距離"]
    df_bike = pd.DataFrame(tmp_df["バイク置き場"].apply(lambda x:feat(x)).apply(pd.Series))
    df_bike.columns = ["バイク置き場_有無", "バイク置き場_円", "バイク置き場_距離"]
    df_cycle = pd.DataFrame(tmp_df["駐輪場"].apply(lambda x:feat(x)).apply(pd.Series))
    df_cycle.columns = ["駐輪場_有無", "駐輪場_円", "駐輪場_距離"]
    
    df_out = pd.concat([df_car, df_bike, df_cycle], axis=1)
    df_out["parking_sum_yen"] = df_out[["駐車場_円", "バイク置き場_円", "駐輪場_円"]].sum(axis=1)
    df_out["parking_sum_meter"] = df_out[["駐車場_距離", "バイク置き場_距離", "駐輪場_距離"]].sum(axis=1)
    df_out["parking_sum_exist"] = df_out[["駐車場_有無", "バイク置き場_有無", "駐輪場_有無"]].sum(axis=1)

    
    return df_out

In [13]:
#a[["駐車場_有無", "バイク置き場_有無", "駐輪場_有無"]].sum(axis=1)

In [14]:
#a["バイク置き場_円"].value_counts()

In [15]:
def split_facilities(text):
    tmp_fcl = []
    if type(text)==float:
        return np.nan
    else:
        fac_list = text.split("\t")
        cnt = text.count("\t")
        if cnt==0:
            cnt+=1
        for i in range(cnt):
            tmp_fcl.append(fac_list[i].split("】")[0].split("【")[1])
        return tmp_fcl

all_facilities = []
fac_val = all_df["周辺環境"].apply(lambda x:split_facilities(x))
for val in fac_val:
    if type(val)==float:
        continue
    all_facilities += val

In [16]:
def around_facilities_feature(df_input):
    df_out = pd.DataFrame()
    def split_facilities(text):
        tmp_fcl = []
        if type(text)==float:
            return np.nan
        else:
            fac_list = text.split("\t")
            cnt = text.count("\t")
            if cnt==0:
                cnt+=1
            for i in range(cnt):
                tmp_fcl.append(fac_list[i].split("】")[0].split("【")[1])
            return tmp_fcl

    all_facilities = []
    fac_val = df_input["周辺環境"].apply(lambda x:split_facilities(x))
    for val in fac_val:
        if type(val)==float:
            continue
        all_facilities += val
    
    all_facilities = list(set(all_facilities))
    
    for fcl in all_facilities:
        df_out[f"has_{fcl}"] = np.where(df_input["周辺環境"].str.contains(fcl), 1, 0)
    df_out["num_of_facilities_type"] = df_out.values.sum(axis=1)
    
    
    dic = {}    
    for i, fcl in enumerate(list(set(all_facilities))):
        dic[fcl] = i
    
    def meter_of_facilities(text, dic=dic):
        ans = np.full(19, np.nan)
        if type(text)==float:
            return pd.Series(ans)
        else:
            for fcl in all_facilities:
                if "【" + fcl + "】" not in text:
                    continue
                else:
                    cnt = text.count("【" + fcl + "】")
                    minim = 100000000
                    #print(fcl)
                    for i in range(1,cnt+1):
                        #print(text.split("【" + fcl + "】"))
                        minim = min(int(text.split("【" + fcl + "】")[i].split("m")[0]), minim)
                ans[dic[fcl]] = minim
            return pd.Series(ans)
    
    meter_df = df_input["周辺環境"].apply(lambda x:meter_of_facilities(x))
    meter_df.columns = list(dic.keys())
    
    meter_df["min_facilities"] = meter_df.min(axis=1)
    meter_df["max_facilities"] = meter_df.max(axis=1)
    meter_df["mean_facilities"] = meter_df.mean(axis=1)
    meter_df["std_facilities"] = meter_df.std(axis=1)
    
    df_out = pd.concat([df_out, meter_df], axis=1)
    
    return df_out

In [17]:
def structure_feature(df_input):
    df_out = pd.DataFrame()
    strc_dict = {'ブロック':1, '鉄筋ブロック':2, '木造':3, '軽量鉄骨':4, 'ALC（軽量気泡コンクリート）':5,
             '鉄骨造':6, 'RC（鉄筋コンクリート）':7, 'SRC（鉄骨鉄筋コンクリート）':8,
             'HPC（プレキャスト・コンクリート（重量鉄骨））':9, 'PC（プレキャスト・コンクリート（鉄筋コンクリート））':10, 'その他':np.nan}
    
    df_out["int建物構造"] = df_input["建物構造"].map(strc_dict)
    
    return df_out

In [18]:
def direction_feature(df_input):
    """価格の高い順に値振り分け　南は高いらしい"""
    df_out = pd.DataFrame()
    direct_dict= {'南':8, '東':5, '西':4, '南東':7, '南西':6, '北':1, '北西':2, '北東':3}
    df_out["int方角"] = df_input["方角"].map(direct_dict)
    return df_out

In [19]:
def built_floor_feature(df_input):
    """
    所在階の中に{／}を含まないものの中から、現実的な長さのものを一軒家とみなす
    階層情報から、低層、高層、タワーマンションで分割
    
    """
    
    search_house = ["1階建", "2階建", "3階建", "4階建", "2階建（地下1階）", "3階建（地下1階）"] 
    df_out = pd.DataFrame()
    df_out["living_floor"] = df_input["所在階"].str.split('／', expand=True)[0].str.split("（", expand=True)[0].str.replace('[^0-9]', '').fillna(0)
    cnt = -1
    for i in df_out["living_floor"]:
        cnt += 1
        if len(str(i)) == 0:
            df_out["living_floor"][df_out["living_floor"].index[cnt]]=np.nan
            
    df_out["living_floor"] = df_out["living_floor"].astype(float)
    
    a = df_input["所在階"].str.split('／', expand=True)[0]
    df_out["一戸建て"] = np.where(a.str.contains('|'.join(search_house)), 1, 0)
    
    df_out["building_floor"] = df_input["所在階"].str.split('／', expand=True)[1].str.split("（", expand=True)[0].str.replace('[^0-9]', '').fillna(0)
    df_out["building_floor"] = df_out["building_floor"].astype(int)
    
    df_out["cat_building_height"] = np.full_like(df_input["所在階"], np.nan)   
    df_out["cat_building_height"] = np.where(df_out["building_floor"] <= 2, 1, df_out["cat_building_height"])
    df_out["cat_building_height"] = np.where((3 <= df_out["building_floor"]) & (df_out["building_floor"] <= 5), 2, df_out["cat_building_height"])
    df_out["cat_building_height"] = np.where((6 <= df_out["building_floor"]) & (df_out["building_floor"] <= 19), 3, df_out["cat_building_height"])
    df_out["cat_building_height"] = np.where(20 <= df_out["building_floor"], 4, df_out["cat_building_height"])
    df_out["cat_building_height"] = df_out["cat_building_height"].astype(str)
    
    df_out["living/building"] = df_out["living_floor"] / df_out["building_floor"]
    df_out["living/building"][df_out["living/building"]==np.inf] = np.nan
    
    df_out["最上階"] = np.where(df_out["living_floor"]==df_out["building_floor"], 1, 0)
    
    df_count = pd.concat([count_encoding(df_out, column) for column in df_out.columns if column not in ["一戸建て"]], axis=1)
    df_out = pd.concat([df_out,df_count], axis=1)
    return df_out

In [20]:
def madori_feature(df_input):
    """間取りの数字部分と、アルファベット部分でそれぞれ特徴量化、納戸フラグ"""
    df_out = pd.DataFrame()
    s = df_input["間取り"]
    df_out["int間取り"] = [int(re.sub('[^0-9]', "", i)) for i in s]
    df_out["cat_int間取り"] = np.where(df_out["int間取り"] > 4, 5, df_out["int間取り"])
    df_out["cat_int間取り"] = df_out["cat_int間取り"].astype(str)
    df_out["納戸"] = [1 if "納戸" in i else 0 for i in s]
    madori_dict = {"R":1, "K":2, "DK":3, "LDK":4, "LK":4}
    df_out["間取りtype"] = [re.sub('[0-9S(納戸)\+]', "", i) for i in s]
    df_out["間取りtype"] = df_out["間取りtype"].map(madori_dict)
    
    df_count = pd.concat([count_encoding(df_out, column) for column in df_out.columns if column not in ["納戸","cat_int間取り"]], axis=1)
    df_out = pd.concat([df_out,df_count], axis=1)
    
    return df_out

In [21]:
def built_year_feature(df_input):
    """築年数を年と月のnumericな特徴に分ける、新築フラグ"""
    df_out = pd.DataFrame()
    df_out["新築"] = df_input["築年数"].values
    idx = df_input["築年数"] == "新築"
    df_out["新築"] = 0
    df_out["新築"][idx] = 1
    
    s = df_input["築年数"]
    df_out["int築年"] = [int(i.split('年')[0]) if "新築" not in i else 0 for i in s]
    df_out["int築月"] = [int(i.split('年')[1].split('ヶ月')[0]) if "新築" not in i else 0 for i in s]
    
    df_out["築月数"] = df_out["int築年"]*12 + df_out["int築月"]
    
    df_count = pd.concat([count_encoding(df_out, column) for column in df_out.columns if column not in ["新築"]], axis=1)
    df_out = pd.concat([df_out,df_count], axis=1)
    
    return df_out

In [22]:
def area_feature(df_input):
    df_out = pd.DataFrame()
    s = df_input["面積"]
    df_out["float面積"] = [float(i.split("m")[0]) for i in s]
    return df_out

In [23]:
old_df = feather.read_dataframe("../features/exp0_+address_poisson_outlin_df.feather")
tmp_df = area_feature(all_df)

In [24]:
def make_value_count_df(df_input, c):
    """与えられた dataframe に対して df_all の count を外部結合する count encoding
    """
    _df = df_input.groupby(c).size().reset_index()
    _df = _df.rename(columns={ 0: f'count_{c}' })

    _df = pd.merge(df_input[[c]], _df, on=c, how='left').drop(columns=[c])
    return _df

def value_count_feature(df_input):
    """Count Encoding を行う特徴量
    一旦全部のカラムを対象にする
    """
    df_out = pd.DataFrame()
    for c in df_input.columns:
        _df = make_value_count_df(df_input, c)
        df_out = pd.concat([df_out, _df], axis=1)
    df_out = df_out.fillna(0)
    return df_out

In [25]:
URL = 'http://www.geocoding.jp/api/'


def coordinate(address):
    """
    addressに住所を指定すると緯度経度を返す。

    >>> coordinate('東京都文京区本郷7-3-1')
    ['35.712056', '139.762775']
    """
    payload = {'q': address}
    html = requests.get(URL, params=payload)
    soup = BeautifulSoup(html.content, "html.parser")
    if soup.find('error'):
        try:
            address = address.split("(")[0] + address.split(")")[1]
            payload = {'q': address}
            html = requests.get(URL, params=payload)
            soup = BeautifulSoup(html.content, "html.parser")
            if soup.find('error'):
                latitude = "-999.0"
                longitude = "-999.0"
            else:
                latitude = soup.find('lat').string
                longitude = soup.find('lng').string
        
        except:
            latitude = "-999.0"
            longitude = "-999.0"
        #raise ValueError(f"Invalid address submitted. {address}")
    else:
        latitude = soup.find('lat').string
        longitude = soup.find('lng').string
    return (latitude, longitude)


def coordinates(addresses, interval=5, progress=True):
    """
    addressesに住所リストを指定すると、緯度経度リストを返す。

    >>> coordinates(['東京都文京区本郷7-3-1', '東京都文京区湯島３丁目３０−１'], progress=False)
    [['35.712056', '139.762775'], ['35.707771', '139.768205']]
    """
    coordinates = {}
    for address in progress and tqdm(addresses) or addresses:
        coordinates[address] = (coordinate(address))
        time.sleep(interval)
    return coordinates

In [26]:
def address_feature(df_input):
    '''
    所在地を区までで区切って、geocoderで緯度経度を求める
    一旦緯度経度のtupleを辞書型にして、その後mapで特徴作り
    '''
    df_out = pd.DataFrame()
    s = df_input["所在地"]
    
    df_out["所在_区"] = [i.split('区')[0] + '区' for i in s]
    
    tmp = {}
    """
    for i in list(set(df_out["所在_区"].values)):
        ret = geocoder.osm(i, timeout=5.0)
        ido, kedo = ret.latlng
        tmp[i] = (ido,kedo)
        
    df_out["緯度経度"] = df_out["所在_区"].map(tmp)
    
    df_out["緯度"] = [i[0] for i in df_out["緯度経度"]]
    df_out["経度"] = [i[1] for i in df_out["緯度経度"]]"""
    
    
    address_dict = coordinates(list(set(df_out["所在_区"].values)), progress=False)
    temp = pd.DataFrame(address_dict).T
    temp.columns = ["緯度", "経度"]
    df_input["区緯度"] = df_out["所在_区"].map(temp.to_dict()["緯度"]).astype(float)
    df_input["区経度"] = df_out["所在_区"].map(temp.to_dict()["経度"]).astype(float)
    
    val = df_input[["区緯度", "区経度"]].values
    clf = GMM(n_components=3)
    clf.fit(val)
    newcol = [f"区_GMM{i}" for i in range(3)]
    GMM_df = pd.DataFrame(clf.predict_proba(val), columns=newcol)
    
    
    count_dict = df_out["所在_区"].value_counts().to_dict()
    df_out["所在_区_count"] = df_out["所在_区"].map(count_dict)

    
    """区以下の特徴量"""
    modify_address = ["東京都世田谷区太子堂一丁目", "東京都大田区本羽田一丁目", "東京都品川区東品川四丁目",
                 "東京都大田区大森北一丁目", "東京都目黒区八雲二丁目"]
    chome = ["1丁目", "1丁目", "4丁目", "1丁目", "2丁目"]

    for add_val, chome_val in zip(modify_address, chome):
        all_df["所在地"] = np.where(all_df["所在地"].str.contains(add_val), all_df["所在地"].str.replace(add_val, chome_val), all_df["所在地"])
    
    def make_chome(text):
        if "丁目" in text:
            return text
        else:
            text = text.replace("-", "丁目", 1)
            return text
    
    val = all_df["所在地"].apply(lambda x:make_chome(x))
    
    df_out["地域名"] = val.str.split("[０-９0-9]", expand=True)[0]
    df_out["count_地域名"] = df_out["地域名"].map(df_out["地域名"].value_counts().to_dict())
    
    df_out["地域_n丁目"] = val.str.split("丁目", expand=True)[0]
    df_out["count_地域_n丁目"] = df_out["地域_n丁目"].map(df_out["地域_n丁目"].value_counts().to_dict())
    
    df_out = pd.concat([df_out, GMM_df], axis=1)
    
    return df_out

In [27]:
def contract_period_feature(df_input):
    """契約期間をnumericに"""
    df_out = pd.DataFrame()
    df_out["定期借家"] = np.where(df_input["契約期間"].str.contains("定期借家", na=False) , 1, 0)
    
    contract = []
    for values in df_input["契約期間"]:
        period = np.inf
        try:
            if "まで" in values:
                year = int(values.split("まで")[0].split("年")[0]) - 2019
                month = int(values.split("まで")[0].split("年")[1].split("月")[0]) - 8
                period = year * 12 + month
            if "ヶ月間" in values:
                if "年" in values:
                    year = int(values.split("ヶ月間")[0].split("年")[0])
                    month = int(values.split("ヶ月間")[0].split("年")[1])
                    period = year * 12 + month
                else:
                    period = int(values.split("ヶ月間")[0])
            if "年間" in values:
                year = int(re.sub("[^0-9]", "", values))
                period = year * 12
        except:
            period = np.nan
        
        contract.append(period)
    df_out["contract_period"] = contract
    
    return df_out

In [28]:
feature_extractors = [
    value_count_feature,
    area_feature,
    built_year_feature,
    address_feature,
    built_floor_feature,
    madori_feature,
    contract_period_feature,
    direction_feature,
    element_count_feature,
    structure_feature,
    around_facilities_feature,
    parking_feature
]

In [29]:
all_df_feature = pd.concat([f(all_df) for f in tqdm(feature_extractors)], axis=1)

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
all_df_feature = all_df_feature.drop(["count_id"], axis=1)
output_dir = '../code/feature_csv/'
all_df_feature.to_feather(os.path.join(output_dir, 'base_feature1.feather'))

In [31]:
list(all_df_feature.columns)

['count_所在地',
 'count_アクセス',
 'count_間取り',
 'count_築年数',
 'count_方角',
 'count_面積',
 'count_所在階',
 'count_バス・トイレ',
 'count_キッチン',
 'count_放送・通信',
 'count_室内設備',
 'count_駐車場',
 'count_周辺環境',
 'count_建物構造',
 'count_契約期間',
 'float面積',
 '新築',
 'int築年',
 'int築月',
 '築月数',
 'count_int築年',
 'count_int築月',
 'count_築月数',
 '所在_区',
 '所在_区_count',
 '地域名',
 'count_地域名',
 '地域_n丁目',
 'count_地域_n丁目',
 '区_GMM0',
 '区_GMM1',
 '区_GMM2',
 'living_floor',
 '一戸建て',
 'building_floor',
 'cat_building_height',
 'living/building',
 '最上階',
 'count_living_floor',
 'count_building_floor',
 'count_cat_building_height',
 'count_living/building',
 'count_最上階',
 'int間取り',
 'cat_int間取り',
 '納戸',
 '間取りtype',
 'count_int間取り',
 'count_間取りtype',
 '定期借家',
 'contract_period',
 'int方角',
 'num_of_NULL',
 '周辺環境_element_counts',
 '室内設備_element_counts',
 'バス・トイレ_element_counts',
 'キッチン_element_counts',
 '放送・通信_element_counts',
 '設備和',
 'count_num_of_NULL',
 'count_周辺環境_element_counts',
 'count_室内設備_element_counts',
 'count_バス・トイレ_ele